In [ ]:
merged_df = pd.merge(ratings_df, movies_df, on='movieId')
merged_df.head()

,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,2,3.5,Jumanji (1995),Adventure|Children|Fantasy
2,1,3,3.0,Grumpier Old Men (1995),Comedy|Romance
3,2,1,5.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,2,4,4.0,Waiting to Exhale (1995),Comedy|Drama|Romance


In [ ]:
user_item_matrix = merged_df.pivot(index='userId', columns='title', values='rating').fillna(0)
user_item_matrix.head()

title,Father of the Bride Part II (1995),Grumpier Old Men (1995),Jumanji (1995),Toy Story (1995),Waiting to Exhale (1995)
userId,,,,,
1,0.0,3.0,3.5,4.0,0.0
2,0.0,0.0,0.0,5.0,4.0
3,3.0,0.0,4.0,0.0,0.0
4,0.0,3.5,0.0,0.0,0.0
5,5.0,0.0,0.0,4.5,0.0


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)
display(user_similarity_df)

userId,1,2,3,4,5
userId,,,,,
1,1.000000,0.511770,0.458770,0.491539,0.438430
2,0.511770,1.000000,0.000000,0.000000,0.522374
3,0.458770,0.000000,1.000000,0.000000,0.445976
4,0.491539,0.000000,0.000000,1.000000,0.000000
5,0.438430,0.522374,0.445976,0.000000,1.000000


In [ ]:
def get_recommendations(user_id, user_item_matrix, user_similarity_df, num_recommendations=5):
    """
    Generates movie recommendations for a user based on collaborative filtering.

    Args:
        user_id: The ID of the user for whom to generate recommendations.
        user_item_matrix: The user-item matrix.
        user_similarity_df: The user-user similarity DataFrame.
        num_recommendations: The number of recommendations to generate.

    Returns:
        A list of recommended movie titles.
    """
    # Get the similarity scores for the given user
    user_similarity_scores = user_similarity_df.loc[user_id]

    # Sort similar users in descending order
    sorted_similar_users = user_similarity_scores.sort_values(ascending=False)

    # Get the movies rated by the target user
    user_rated_movies = user_item_matrix.loc[user_id][user_item_matrix.loc[user_id] > 0].index

    # Initialize a dictionary to store predicted ratings
    predicted_ratings = {}

    # Iterate through similar users (excluding the target user)
    for similar_user_id in sorted_similar_users.index:
        if similar_user_id != user_id:
            # Get movies rated by the similar user
            similar_user_rated_movies = user_item_matrix.loc[similar_user_id][user_item_matrix.loc[similar_user_id] > 0].index

            # Identify movies rated by the similar user but not by the target user
            unrated_movies = similar_user_rated_movies.difference(user_rated_movies)

            for movie in unrated_movies:
                # Get the rating of the similar user for this movie
                similar_user_rating = user_item_matrix.loc[similar_user_id, movie]

                # Get the similarity score between the target user and the similar user
                similarity_score = sorted_similar_users.loc[similar_user_id]

                # Calculate weighted rating and add to predicted_ratings
                if movie not in predicted_ratings:
                    predicted_ratings[movie] = 0
                predicted_ratings[movie] += similar_user_rating * similarity_score

    # Sort unrated movies by predicted ratings
    sorted_recommendations = sorted(predicted_ratings.items(), key=lambda item: item[1], reverse=True)

    # Return the top N recommended movies
    return [movie for movie, rating in sorted_recommendations[:num_recommendations]]

# Example usage of the function
user_id_to_recommend = 1
recommendations = get_recommendations(user_id_to_recommend, user_item_matrix, user_similarity_df)
print(f"Recommendations for user {user_id_to_recommend}: {recommendations}")

Recommendations for user 1: ['Father of the Bride Part II (1995)', 'Waiting to Exhale (1995)']
